In [ ]:
%load_ext autoreload
%autoreload 1

In [1]:
import html2text
import httpx
import json
import lxml
import re
import glob

from lxml import html
from extractcontent3 import ExtractContent
from readability.readability import Document

from tqdm import tqdm

In [2]:
def extractcontent(html):
    extractor = ExtractContent()

    opt = {"threshold":50}
    extractor.set_option(opt)

    extractor.analyse(html)
    text, title = extractor.as_text()
    html, title = extractor.as_html()
    title = extractor.extract_title(html)
    return text

def readability(html):
    doc = Document(html)
    title = doc.title()
    summary = doc.summary()
    #text1 = html2text.html2text(article)
    summary_text = lxml.html.fromstring(summary).text_content().strip()
    #return text1 + '\n---\n' + text2
    return title, summary_text

In [3]:
!ls fc2blog/

1    120  142  164  186  207  229  250	272  294  315  337  36	 381  59  80
10   121  143  165  187  208  23   251	273  295  316  338  360  382  6   81
100  122  144  166  188  209  230  252	274  296  317  339  361  39   60  82
101  123  145  167  189  21   231  253	275  297  318  34   362  4    61  83
102  124  146  168  19	 210  232  254	276  298  319  340  363  40   62  84
103  125  147  169  190  211  233  255	277  299  32   341  364  41   63  85
104  126  148  17   191  212  234  256	278  3	  320  342  365  42   64  86
105  127  149  170  192  213  235  257	279  30   321  343  366  43   65  87
106  128  15   171  193  214  236  258	28   300  322  344  367  44   66  88
107  129  150  172  194  215  237  259	280  301  323  345  368  45   67  89
108  13   151  173  195  216  238  26	281  302  324  346  369  46   68  9
109  130  152  174  196  217  239  260	282  303  325  347  37	 47   69  90
11   131  153  175  197  218  24   261	283  304  326  348  370  48   7   91
110  132  154  176

In [4]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36'
}
cookies = {
    'age_check': '1'
}

In [5]:
toxicity_genres = (333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 378, 379, 381)

In [6]:
genre_list = httpx.get('https://blog.fc2.com/genrelist/', headers=headers, cookies=cookies)
genre_groups = html.fromstring(genre_list.text).xpath('//*[@class="genre_group"]')
genre_data = []
for group in genre_groups:
    genre = group.xpath('./h3/a')[0]
    genre_id = genre.attrib['href'].strip().split('/')[-2]
    genre_text = genre.text.strip()
    
    children = []
    subgenres = group.xpath('./ul/li/a')
    for node in subgenres:
        subgenre_id = re.findall(r'/subgenre/([0-9]+)/', node.attrib['href'])[0]
        subgenre_text = ''.join(node.itertext()).strip()
        children.append({
            'id': subgenre_id,
            'name': subgenre_text
        })

    genre_data.append({
        'id': genre_id,
        'name': genre_text,
        'subgenre': children
    })
    
subgenre2genre = {subgenre['id']: (genre['id'], genre['name']) for genre in genre_data for subgenre in genre['subgenre']}
subgenre2name = {subgenre['id']: subgenre['name'] for genre in genre_data for subgenre in genre['subgenre']}

In [7]:
data = []
for path in tqdm(glob.glob('./fc2blog/*/*.html')[:10]):
    entry = path.split('/')[-1]
    subgenre_id = path.split('/')[2]
    toxicity = 1 if genre_id in toxicity_genres else 0
    try:
        title, summary = readability(open(path).read())
        if summary.strip():
            data.append({
                'blog': entry.split('#')[0],
                'entry': entry.split('#')[1],
                'genre_id': subgenre2genre[subgenre_id][0],
                'genre_name': subgenre2genre[subgenre_id][1],
                'subgenre_id': subgenre_id,
                'subgenre_name': subgenre2name[subgenre_id],
                'toxicity': toxicity,
                'title': title,
                'text': summary
            })
    except Exception as exc:
        print(f"{type(exc).__name__}. {str(exc)} {path}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 78.32it/s]

ParserError. Document is empty ./fc2blog/1/springillustration.blog.fc2.com#blog-entry-1020.html
ParserError. Document is empty ./fc2blog/1/oyamanosuginoki.blog.fc2.com#blog-entry-2904.html
ParserError. Document is empty ./fc2blog/1/watcher184.blog.fc2.com#blog-entry-88.html
ParserError. Document is empty ./fc2blog/1/himawarigakudouclub.blog.fc2.com#blog-entry-745.html


In [8]:
data[0]

{'blog': 'create39.blog117.fc2.com',
 'entry': 'blog-entry-4267.html',
 'genre_id': '7',
 'genre_name': '日記',
 'subgenre_id': '1',
 'subgenre_name': '小学生',
 'toxicity': 0,
 'title': '大好きなヤマちゃん - ～create～ hiroko 日々のフォトライフ',
 'text': 'セイタカアワダチソウを写して、帰ろうとしたその時、ヤマセミヤマちゃんのキャッキャッと鳴き声が聞こえて来ました。それも近い所から・・・・・静かにそぉ～と動きながら探したら、「居ましたぁ～」遮る枝を避けて写そうとしたら・・・・２回シャッターを切ったら飛ばれました。大好きなヤマちゃんに会えて、スイッチが入りいつも遭遇する確率の高いポイントにUターンしてしばらく待つことにしました。見えない所で鳴き声がしているので、必ず来ると信じて待ちました。やっとこ来てくれたけれど、枝に遮られて・・・・・直ぐに飛んで行きました。もう一度だけ、待っていたら、少し高い枝に来てくれました。待っていて良かったぁ～\u3000\u3000\u3000本当に嬉しかったです。ちょっと移動して、こっちを向いています。\u3000\u3000正面顔です。又、飛んでいったので、今度こそ帰ろう～と移動したら先々、止まっては鳴き・・・・でも、これ写して後ろ髪引かれながらも、山の１６時は薄暗く成り始めこの場を後にしました。'}

In [9]:
with open('/tmp/fc2blog.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)